In [10]:
from typing import List
import pandas as pd

In [2]:
dvs_data = pd.read_excel('./data/ChemRev_Hydration_Data.xlsx', sheet_name='HydrationData',usecols = 'A:R')   #importing experimental data

/Users/danielschweigert/PycharmProjects/2024-ionomer-analysis/venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/danielschweigert/PycharmProjects/2024-ionomer-analysis/venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [3]:
dvs_data.head()

,Study,Year,Membrane,EW,Cast,Cast_id,Thickness,PT,Drying_Temp,Pret_Time,Temperature,LV,LV_id,RH_r,RH,lambda,ChemPotential,Equilibrium_hr
0,Alberti,2008,N117,1100,Ext,1,175.0,1,120,15.0,20.0,L,1.0,100.1,100.1,12.4,11220.638568,1.0
1,Alberti,2008,N117,1100,Ext,1,175.0,1,120,15.0,40.0,L,1.0,100.1,100.1,13.2,11986.552464,1.0
2,Alberti,2008,N117,1100,Ext,1,175.0,1,120,15.0,60.0,L,1.0,100.1,100.1,14.0,12752.466359,1.0
3,Alberti,2008,N117,1100,Ext,1,175.0,1,120,15.0,70.0,L,1.0,100.1,100.1,12.7,13135.423307,1.0
4,Alberti,2008,N117,1100,Ext,1,175.0,1,120,15.0,80.0,L,1.0,100.1,100.1,15.4,13518.380255,1.0


### Filter down to relevant data

In [4]:
# desciption: why filtered out: ...

not_include = ['Cast','LV','RH_r','Pret_Time']
df = dvs_data.query('Study != "Alberti" and EW == 1100')
df = dvs_data.drop(not_include, axis=1)
df = df[df['Thickness'].notnull()]

In [6]:
# subset 1
# description: remove equilibrium_hr column, divide in liquid and vapor
df_1_columns = df.columns.values.tolist()
df_1_columns.remove('Equilibrium_hr')
df_1 = df[df_1_columns]
print('dataset 1: ', df_1.shape)
df_1_l = df_1[df_1['LV_id']==1]
df_1_l['RH'] = df_1_l['RH'].replace(100.1,100)
df_1_v = df_1[df_1['LV_id']==0]
print('dataset 1_l: ', df_1_l.shape)
print('dataset 1_v: ', df_1_v.shape)

dataset 1:  (784, 13)
dataset 1_l:  (242, 13)
dataset 1_v:  (505, 13)


/var/folders/bx/gzbw6rqj1_dg2cswsj6km8_80000gq/T/ipykernel_64029/3935855.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_l['RH'] = df_1_l['RH'].replace(100.1,100)


In [7]:
# subset 2
# description: non-null Equilibrium_hr data, divide in liquid and vapor
df_2 = df[df['Equilibrium_hr'].notnull()]
print('dataset 2: ', df_2.shape)
df_2_l = df_2[df_2['LV_id']==1]
df_2_v = df_2[df_2['LV_id']==0]
print('dataset 2_l: ', df_2_l.shape)
print('dataset 2_v: ', df_2_v.shape)

dataset 2:  (617, 14)
dataset 2_l:  (215, 14)
dataset 2_v:  (365, 14)


### Question 1:
#### Can we fit a model on material properties and processing conditions to predict performance metric delta lambda?

In [9]:
material_properties = ['Thickness']
processing_conditions = ['PT', 'Cast_id', 'Drying_Temp', 'Equilibrium_hr']
operating_conditions = ['Temperature']
predictors = material_properties + processing_conditions + operating_conditions

In [11]:
def compute_performance_metric_delta_lambda(df: pd.DataFrame, predictors: List[str]) -> pd.DataFrame:
    """
    Input:
    DataFrame with columns
        ...predictors..., LV_id, RH, lambda

    Output:
        DataFrame with columns
        ...predictors..., delta_lambda
    """
    ...


In [44]:
def unstack_liquid_vapor_datasets(df: pd.DataFrame, fixed_columns: List[str]) -> pd.DataFrame:
    """
    Input DataFrame:
        ...fixed_columns..., LV_id, lambda

    Output DataFrame:
        ...fixed_columns..., lambda_lv_0, lambda_lv_1

    example for fixed_columns: [EW, Thickness, PT, Cast_id, Drying_Temp, Equilibrium_hr, Temperature, RH]
    """
    df_wide = df.set_index(fixed_columns + ['LV_id']).unstack('LV_id')
    df_wide.columns = [f'{col[0]}_lv_{col[1]}' for col in df_wide.columns]
    df_wide = df_wide.reset_index()
    return df_wide


def test_unstack_liquid_vapor_datasets():

    df = pd.DataFrame(
        data={
            'A': [1, 2, 3, 4, 4],
            'B': [6, 7, 8, 4, 4],
            'LV_id': [0, 1, 0, 1, 0],
            'RH': [50, 100, 90, 100, 100],
            'lambda': [5, 15, 25, 35, 45]
        }
    )

    df_result = unstack_liquid_vapor_datasets(df=df, fixed_columns=['A', 'B', 'RH'])

    print(df)

    print('----')
    print(df_result)


test_unstack_liquid_vapor_datasets()



   A  B  LV_id   RH  lambda
0  1  6      0   50       5
1  2  7      1  100      15
2  3  8      0   90      25
3  4  4      1  100      35
4  4  4      0  100      45
----
   A  B   RH  lambda_lv_0  lambda_lv_1
0  1  6   50          5.0          NaN
1  2  7  100          NaN         15.0
2  3  8   90         25.0          NaN
3  4  4  100         45.0         35.0
